In [1]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import load_model

2022-08-22 14:56:05.463973: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-22 14:56:05.463994: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


### Sprawdzic SSIM bez sieci dla przykładowych zdjęć
https://ourcodeworld.com/articles/read/991/how-to-calculate-the-structural-similarity-index-ssim-between-two-images-with-python

In [2]:
from tqdm import tqdm
test_path = "../data/chest_xray/test"
dsize =(64, 64)

labels = {"NORMAL":0, "PNEUMONIA":1}

def load_images(dir_path, labels, dsize = (32, 32)):
    """loading images"""
    X_image, y_image = [], []
    for label_name, Id in labels.items():
        for jpeg in tqdm(os.listdir(os.path.join(dir_path, label_name))):
            path = os.path.join(dir_path, label_name, jpeg)
            image = load_img(path, target_size=dsize)
            X_image.append(image)
            #self.X_image.append(img_to_array(image)/255.0)
            y_image.append(Id)
    return X_image, y_image

X_test_org, y_test = load_images(test_path, labels, dsize = dsize)

100%|██████████| 234/234 [00:01<00:00, 131.41it/s]
0it [00:00, ?it/s]


In [3]:
import random
def preprocess_images(images, y, normalize = True):
    X = []
    for i in tqdm(range(len(images))):
        img = img_to_array(images[i]).astype('float32')
        if normalize:
            img = img/255.0
        X.append(img)
    #Shuffle data
    dataset = list(zip(X, y))
    random.shuffle(dataset)

    X, y = zip(*dataset)
    return np.array(X), np.array(y)

normalize = True
x_test, y_test =preprocess_images(X_test_org, y_test, normalize=normalize)
print("Shape of X_test:", x_test.shape)

100%|██████████| 234/234 [00:00<00:00, 25990.18it/s]

Shape of X_test: (234, 64, 64, 3)


In [4]:
def SSIMLoss(y_true, y_pred):
  return 1 - tf.reduce_mean(tf.image.ssim(y_true, y_pred,1.0))

In [5]:
# models/pneumonia_ae_acc77ssim06.h5
model = load_model('/home/iotg/work/SWA/SWA/models/pneumonia_ae_acc77ssim06.h5', custom_objects={'SSIMLoss': SSIMLoss})

2022-08-22 14:57:11.474944: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-08-22 14:57:11.474970: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (iotg): /proc/driver/nvidia/version does not exist
2022-08-22 14:57:11.475185: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 64, 64, 3)]       0         
                                                                 
 conv_1 (Conv2D)             (None, 64, 64, 32)        896       
                                                                 
 batchnorm_1 (BatchNormaliza  (None, 64, 64, 32)       128       
 tion)                                                           
                                                                 
 leaky_relu_1 (LeakyReLU)    (None, 64, 64, 32)        0         
                                                                 
 conv_2 (Conv2D)             (None, 32, 32, 64)        18496     
                                                                 
 batchnorm_2 (BatchNormaliza  (None, 32, 32, 64)       256       
 tion)                                                       

In [7]:
import gradio as gr

In [18]:
def classify_image(inp):
    inp = inp.resize((64, 64))
    img_array = img_to_array(inp).astype('float32')
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array/255.0
    model = load_model('/home/iotg/work/SWA/SWA/models/pneumonia_ae_acc77ssim06.h5', custom_objects={'SSIMLoss': SSIMLoss})
    prediction = model.predict(img_array)
    output = SSIMLoss(prediction, img_array)
    #prediction = np.squeeze(prediction, axis=0)
    #return prediction
    if output > 0.6:
        return "ZDROWE", output
    else: 
        #return np.max(inp), np.max(img_array), img_array.shape, output
        return "Zapalenie płuc", output

In [20]:
gr.close_all()


Closing server running on port: 7861
Closing server running on port: 7861
Closing server running on port: 7861


In [19]:
gr.Interface(fn=classify_image, 
             inputs=gr.Image(type="pil"),
             outputs="text"
             #examples=["/home/iotg/work/SWA/SWA/data/chest_xray/val/NORMAL/NORMAL2-IM-1427-0001.jpeg"])
             ).launch(server_port=7861 , share=True, debug=True)

Running on local URL:  http://127.0.0.1:7861/
*** Failed to connect to ec2.gradio.app:22: [Errno 110] Connection timed out


AssertionError: 